In [1]:
#!/usr/bin/env python3

import sys
from scapy.all import *
from time import *

import statistics
from statistics import mode, mean
import pandas as pd

## Definiciones

In [2]:

def isTimeExceeded(pkt):
    return pkt[ICMP].type == 11

def rttsXX(url, repCount, ttlRange):
    responses = {}
    for i in range(repCount):
        for ttl in ttlRange:
            probe = IP(dst=url, ttl=ttl) / ICMP()
            t_i = time()
            ans = sr1(probe, verbose=False, timeout=0.8)
            t_f = time()
            rtt = (t_f - t_i)*1000
            if (ans is not None) and isTimeExceeded(ans):

                if ttl not in responses: 
                    responses[ttl] = []
                responses[ttl].append((ans.src, rtt))
        #print(responses)
        #print()    
    return responses

def mostFrequentIPSAndTheirMeanRTTs(rtts):
    mostFrequentIPs = { key : mode([ p[0] for p in value ]) for key, value in rtts.items() }
    
    filteredResponsesByMostFrequent = { key : [x for x in value if x[0]==mostFrequentIPs[key]] for key, value in rtts.items() }
    
    meanRTTs = { key : mean([ p[1] for p in value ]) for key, value in filteredResponsesByMostFrequent.items() }
    
    return mostFrequentIPs, meanRTTs

def getStepsXX(RTTsByTTL, mostFrequentIPs):
    totalRTTs = RTTsByTTL.copy()
    RTTsOfSteps = {}
    IPsOfSteps = mostFrequentIPs.copy()
    sortedKeys = sorted(RTTsByTTL.keys())
    for idx, TTL in enumerate(sortedKeys):
        if idx == 0:
            RTTsOfSteps[TTL] = RTTsByTTL[TTL]
        else:
            for backwardsSkip in range(1,idx+1):
                if (idx - backwardsSkip == 0):
                    diff = RTTsByTTL[TTL] 
                else:
                    previousTTL = sortedKeys[idx-backwardsSkip]
                    if previousTTL not in RTTsOfSteps:
                        continue
                    diff = RTTsByTTL[TTL] - RTTsByTTL[previousTTL]
                if diff > 0:
                    break
                RTTsOfSteps.pop(previousTTL)
                IPsOfSteps.pop(previousTTL)
                totalRTTs.pop(previousTTL)
            RTTsOfSteps[TTL] = diff
    return RTTsOfSteps, IPsOfSteps, totalRTTs

def traceroute(url):
    responses = rttsXX(url, 30, range(1,30))
    
    mostFrequentIPs, meanRTTs = mostFrequentIPSAndTheirMeanRTTs(responses)
    
    steps, IPsOfSteps, totalRTTsOfIPs= getStepsXX(meanRTTs, mostFrequentIPs)
    
    return IPsOfSteps, steps, totalRTTsOfIPs

- Universidades:
    - Heidelberg Universität
        www.uni-heidelberg.de
    - 北京大学
        www.pku.edu.cn
    - University of Cape Town
        www.uct.ac.za

In [3]:
universityUrls = { 
    "Heidelberg": "www.uni-heidelberg.de", 
    "Peking": "www.pku.edu.cn", 
    "Cape Town": "www.uct.ac.za" 
}

In [4]:
def runExperiment1(universityUrls):
    
    results = {} 

    for university, url in universityUrls.items():
        
        print(university)
        print(url)
        print()

        ips, stepRTTs, totalRTTs = traceroute(url)

        results[university] = pd.DataFrame([{"TTL": ttl, "IP": ips[ttl], "Step RTT": stepRTTs[ttl], "Total RTT": totalRTTs[ttl]} for ttl in ips.keys()])
    
    return results

In [5]:
def saveExperiment1Results(results, outputFolder, outputFileNameRoot):
    
    for university, trace in results.items():

            outputPath = outputFolder + outputFileNameRoot + "-" + university

            trace.to_csv(outputPath, index=False)

## Experimento y resultados

In [9]:
results = runExperiment1(universityUrls)

for university, trace in results.items():
    
    print(university)
    print(trace)
    print()

Heidelberg
www.uni-heidelberg.de

Peking
www.pku.edu.cn

Cape Town
www.uct.ac.za

Heidelberg
   TTL               IP    Step RTT   Total RTT
0    1      192.168.0.1   15.622616   15.622616
1    7    195.22.220.56   31.266689   31.266689
2    9    89.221.41.199  109.699488  140.966177
3   10   62.115.120.176   46.537399  187.503576
4   18  129.206.216.241   64.589739  252.093315

Peking
    TTL               IP    Step RTT   Total RTT
0     1      192.168.0.1    0.000000    0.000000
1     7    195.22.220.56   31.210184   31.210184
2     8    195.22.219.67    5.946636   37.156820
3    10    129.250.4.202    9.754658   46.911478
4    12    129.250.6.177  171.837568  218.749046
5    13    129.250.4.143   76.897860  295.646906
6    15    129.250.6.123   32.694101  328.341007
7    16  203.131.254.214   14.285326  342.626333
8    18    101.4.118.121   13.758183  356.384516
9    19    101.4.115.113    8.349895  364.734411
10   23   202.112.41.182    6.590128  371.324539
11   24     202.112.41.

In [10]:
outputFolder = "./Resultados/"
outputFileNameRoot = "Experimento1-Fermin"
saveExperiment1Results(results, outputFolder, outputFileNameRoot)